In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import sacrebleu
import sys
import os
sys.path.append("/fs/clip-scratch/sweagraw/contrastive-controlled-mt/IWSLT2022")

In [2]:
def read_file(fname):
    data = []
    with open(fname) as f:
        for line in f:
            data.append(line.strip())
    return data

In [3]:
def get_data(tgt_lang, domain, split):
    source = read_file(f"../internal_split/en-{tgt_lang}/{split}.{domain}.en")
    formal_translations = read_file(f"../internal_split/en-{tgt_lang}/{split}.{domain}.formal.{tgt_lang}")
    informal_translations = read_file(f"../internal_split/en-{tgt_lang}/{split}.{domain}.informal.{tgt_lang}")
    return source, formal_translations, informal_translations

In [4]:
tgt_lang_to_code = {
    "hi" : "hi_IN",
    "de" : "de_DE",
    "es" : "es_XX",
    "it" : "it_IT",
    "ru" : "ru_RU",
    "ja" : "ja_XX"
}

In [5]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", cache_dir="/fs/clip-scratch/sweagraw/CACHE")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX", cache_dir="/fs/clip-scratch/sweagraw/CACHE")

In [7]:
model = MBartForConditionalGeneration.from_pretrained("../models/facebook/mbart-large-50-one-to-many-mmt-finetuned-en-to-xx-informal", cache_dir="/fs/clip-scratch/sweagraw/CACHE")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX", cache_dir="/fs/clip-scratch/sweagraw/CACHE")

In [8]:
def translate_text(text, tgt_lang):
    model_inputs = tokenizer(text, return_tensors="pt", padding=True)

    # translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang_to_code[tgt_lang]]
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [9]:
src_lang = "en"
tgt_lang = "hi"
domain="combined"
split="dev"
source, formal_translations, informal_translations = get_data(tgt_lang, domain, split)

In [10]:
output_dir=f"../experiments/{src_lang}-{tgt_lang}/mBART_informal/{domain}/"
os.makedirs(output_dir, exist_ok=True)

In [11]:
outputs = translate_text(source, tgt_lang)

RuntimeError: [enforce fail at CPUAllocator.cpp:68] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 500108000 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
 with open(output_dir+"/out."+split, "w") as f:
        for out in outputs:
            f.write(out + "\n")

In [30]:
# Other decodng strategies

In [31]:
model_inputs = tokenizer(source[0], return_tensors="pt", padding=True)

In [39]:
# translate from English to Hindi
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang_to_code[tgt_lang]],
    max_length=50, 
    num_beams=5, 
    num_return_sequences=5, 
    early_stopping=True
)

In [40]:
generated_tokens

tensor([[     2, 250010,  10399, 188051,  45389,    421,  24062,   6143,    471,
          33572,   4050,    471,  86990,  18224,  45718,      5,   4239,  21568,
            871, 139041,    871, 219046,    460,    871,  33171,    460,      5,
          15273,   3849,      4,   4239,  32243,   1480,    460,      5, 109440,
            460,   1682,   4239,    460,      5,      2],
        [     2, 250010,  10399, 188051,  45389,    421,  24062,   6143,    471,
          33572,   4050,    471,  86990,  18224,  28035,      5,   4239,  21568,
            871, 139041,    871, 219046,    460,    871,   4239,    460,      5,
          15273,   3849,      4,   4239,  32243,   1480,    460,      5, 109440,
            460,   1682,   4239,    460,      5,      2],
        [     2, 250010,  10399, 188051,  45389,    421,  24062,   6143,    471,
          33572,   4050,    471,  86990,  18224,  45718,      5,   4239,  21568,
            871, 139041,    871, 219046,    460,    871,   4239,    460,  

In [41]:
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['मैं निकट भविष्य में आपके देश की यात्रा करने की आशा करता हूं. यह पूर्ण और सरल और मधुर है और यही है. ओह, यह कहता है. उम्मीद है कि यह है.',
 'मैं निकट भविष्य में आपके देश की यात्रा करने की आशा करता हूँ. यह पूर्ण और सरल और मधुर है और यह है. ओह, यह कहता है. उम्मीद है कि यह है.',
 'मैं निकट भविष्य में आपके देश की यात्रा करने की आशा करता हूं. यह पूर्ण और सरल और मधुर है और यह है. ओह, यह कहता है. उम्मीद है कि यह है.',
 'मैं निकट भविष्य में आपके देश की यात्रा करने की आशा करता हूँ. यह पूर्ण और सरल और मधुर है और यह है. ओह, यह कहता है. यह उम्मीद है.',
 'मैं निकट भविष्य में आपके देश की यात्रा करने की आशा करता हूं. यह पूर्ण और सरल और मधुर है और यही है. ओह, यह कहता है. उम्मीद है यह है.']

# Evaluation Covariates

In [7]:
from transformers import AutoTokenizer
import sacrebleu
import sys
import os
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, MBartForConditionalGeneration, MBart50TokenizerFast

In [8]:
from mbart_covariate import CMBartForConditionalGeneration

In [9]:
model = CMBartForConditionalGeneration.from_pretrained("../models/facebook/mbart-large-50-one-to-many-mmt-finetuned-covariate-en-to-xx", cache_dir="/fs/clip-scratch/sweagraw/CACHE")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX", cache_dir="/fs/clip-scratch/sweagraw/CACHE")

In [10]:
import torch
def translate_text(text, tgt_lang, covariate_index):
    model_inputs = tokenizer(text, return_tensors="pt", padding=True)
    kwargs = {}
    kwargs["covariate_ids"] = torch.tensor([covariate_index]*len(text))

    # translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang_to_code[tgt_lang]],
        **kwargs
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [11]:
outputs = translate_text(source, tgt_lang, covariate_index=1)

In [12]:
outputs

['असल में डिज़्नी के सभी चीजों में मिकी माउस चित्रों को ढूँढ़ने के लिए एक छुपे उपकौशंसल मौजूद है, आपको जरूर देखना चाहिए!',
 'हाँ, बहुत बढ़िया बात है। शायद तुम्हारे कुत्ते दानदाता हो सकते हैं। मुझे उम्मीद है कि ड्राइव पर कुत्तों को बहुत से चखाने मिलेंगे।',
 'सही है, लेकिन मुझे लगता है कि बिल्लियाँ बहुत से राजनीतिज्ञों से ज़्यादा होशियार हैं। क्या आप देखते हैं कि आपका आम हाउस सदस्य लम्बे समय तक अलग रहता है?',
 'यह गारंट नहीं है कि आप खेल जीतेंगे लेकिन यह सुनिश्चित करता है कि यह बहुत आसान हो जाएगा. लोल. एक बढ़िया दिन है.',
 'हाँ यह समझ में आया है, क्या आपने सुना है कि उनके पास 10 मिलियन डॉलर का बांकर है?',
 'हाँ, और ज़्यादातर जगहों पर आप अभी भी अपने खाते पर पहुँच कर सकते हैं क्योंकि यह चीन, सीरिया, उत्तर कोरिया और क्रिमा के अलावा देशों में उपलब्ध है।',
 'मुझे पता है कि क्या आप अतिरिक्त प्रत्यय प्राप्त कर सकते हैं अगर आप उस कक्षा के लिए अपने बाल उगाने शुरू कर देते हैं? लोल मुझे कॉलेज में एक पौराणिक और कल्पना कक्षा थी। यह बहुत अच्छा था।',
 'यह मेरा उपनाउँ था! इसलिए नहीं कि मैं शेक्सपियर की 

In [32]:
outputs

['असल में डिज़्नी के सभी चीजों में मिकी माउस चित्रों को ढूँढ़ने के लिए एक छुपे उपसंस्कृति है, आपको जरूर देखना चाहिए!',
 'हाँ, बहुत बढ़िया बात है। शायद आपके कुत्ते दानदाता हो सकते हैं। मुझे उम्मीद है कि ड्राइव पर कुत्तों को बहुत से चखाने मिलेंगे।',
 'सच है, लेकिन मुझे लगता है कि बिल्लियाँ बहुत से राजनीतिज्ञों की तुलना में ज़्यादा होशियार हैं। क्या आप देखते हैं कि आपका आम सदस्य लम्बे समय तक विभाजन करता है?',
 'यह आपको इस खेल को जीने की गारंटी नहीं देगी लेकिन इससे इसे बहुत आसान हो जाएगा. लोल. एक बढ़िया दिन बिताओ.',
 'हाँ यह समझ में आया है, क्या आपने सुना है कि उनके पास 10 मिलियन डॉलर का बंकर है?',
 'हाँ, और बहुत से स्थानों पर आप अभी भी अपने खाते पर पहुँच कर सकते हैं क्योंकि यह चीन, सीरिया, उत्तर कोरिया और क्रिमा के अलावा देशों में उपलब्ध है।',
 'मुझे पता है कि क्या आप अतिरिक्त प्रत्यय प्राप्त कर सकते हैं अगर आप उस कक्षा के लिए अपने बाल उगाने शुरू कर देते हैं? लोल मुझे कॉलेज में एक पौराणिक और कल्पना कक्षा थी। यह बहुत अच्छा था।',
 'यह मेरा उपनाम था! इसलिए नहीं कि मैं शेक्सपियर की तरह बड़ा ह